# Imports

In [ ]:
import numpy as np
import casadi as cd
from matplotlib import pyplot as plt

You can adapt all plotting parameters below.

In [ ]:
plt.rcParams.update({
    "font.size": 12,
    # "text.usetex": True,
    "text.usetex": False,
})

# Calculation of the policy gradient and Hessian

In [ ]:
# Parameters for the normal distribution of noise w
mu_w = 0.0 # Mean
sigma_w = cd.SX.sym("sigma_w") # Standard deviation

# Parameters for the normal distribution of initial state s0
mu_s0 = 0.0 # Mean
sigma_s0 = cd.SX.sym("sigma_s0") # Standard deviation


# States and actions
s = cd.SX.sym("s") # State
a = cd.SX.sym("a") # Action

# Parameters 
gamma = cd.SX.sym("gamma") # Discount factor

# Policy
theta = cd.SX.sym("theta") # Policy parameter
pi = - theta ** 2 * s # Policy function
jac_pi_theta = cd.jacobian(pi, theta) # Jacobian of the policy with respect to theta
jac2_pi_theta = cd.jacobian(jac_pi_theta, theta) # Second derivative of the policy with respect to theta

# Reward function
reward = -0.5 * (s ** 2 + a ** 2)
reward_func = cd.Function("reward", [s, a], [reward])
reward_at_policy = reward_func(s, pi)

# Value function parameters
p = -1/2 * (1 + theta ** 4) / (1 - gamma * (1 - theta ** 2) ** 2)
q = gamma/(1 - gamma) * sigma_w ** 2 * p

# Derivatives of p and q with respect to theta
grad_p_theta = cd.jacobian(p, theta)
grad_q_theta = cd.jacobian(q, theta)

# Value and Q functions
V = p * s ** 2 + q
V_func = cd.Function("V", [s, theta, gamma, sigma_w], [V], ["s", "theta", "gamma", "sigma_w"], ["V"])
Q = reward + gamma * (q + p * sigma_w ** 2) + gamma * p * (s + a) ** 2
Q_func = cd.Function("Q", [s, a, theta, gamma, sigma_w], [Q], ["s", "a", "theta", "gamma", "sigma_w"], ["Q"])

# Expected cumulative reward
J = q + p * sigma_s0 ** 2
J_func = cd.Function("J", [theta, gamma, sigma_w, sigma_s0], [J], ["theta", "gamma", "sigma_w", "sigma_s0"], ["J"])

# Derivatives of J with respect to theta (these are the exact derivatives)
hess_J, grad_J = cd.hessian(J, theta)
grad_J_func = cd.Function("grad_J", [theta, gamma, sigma_w, sigma_s0], [grad_J], ["theta", "gamma", "sigma_w", "sigma_s0"], ["grad_J"])
hess_J_func = cd.Function("hess_J", [theta, gamma, sigma_w, sigma_s0], [hess_J], ["theta", "gamma", "sigma_w", "sigma_s0"], ["hess_J"])

# Derivatives of the Q function with respect to a
hess_Q_a, grad_Q_a = cd.hessian(Q, a)

grad_Q_a_func = cd.Function("grad_Q_a", [s, a, theta, gamma], [grad_Q_a])
grad_Q_a_at_policy = grad_Q_a_func(s, pi, theta, gamma)
grad_Q_a_at_policy_func = cd.Function("grad_Q_a_at_policy", [s, theta, gamma], [grad_Q_a_at_policy])

hess_Q_a_func = cd.Function("hess_Q_a", [s, a, theta, gamma], [hess_Q_a])
hess_Q_a_at_policy = hess_Q_a_func(s, pi, theta, gamma)
hess_Q_a_at_policy_func = cd.Function("hess_Q_a_at_policy", [s, theta, gamma], [hess_Q_a_at_policy])

# Auxiliary variables to save typing effort
c1 = 2 * theta ** 3 * (2 * gamma * p - 1) - 4 * gamma * p * theta

# Expected value of squared state
expected_value_of_squared_s = (grad_q_theta + sigma_s0 ** 2 * grad_p_theta) / c1

# Gradient of the expected cumulative reward but using the deterministic policy theorem
grad_J_RL = c1 * expected_value_of_squared_s
grad_J_RL_func = cd.Function("grad_J_RL", [theta, gamma, sigma_w, sigma_s0], [grad_J_RL], ["theta", "gamma", "sigma_w", "sigma_s0"], ["grad_J_RL"])

# Gauss-Newton matrix (M2)
Gauss_Newton_matrix = 4 * theta ** 2 * (2 * gamma * p - 1) * expected_value_of_squared_s
Gauss_Newton_matrix_func = cd.Function("Gauss_Newton_matrix", [theta, gamma, sigma_w, sigma_s0], [Gauss_Newton_matrix], ["theta", "gamma", "sigma_w", "sigma_s0"], ["Gauss_Newton_matrix"])

# Approximate Hessian (M1 + M2)
H1 = (2 * theta ** 2 *(2 * gamma * p - 1)- 4 * gamma * p) * expected_value_of_squared_s
H2 = Gauss_Newton_matrix
H = H1 + H2
H_func = cd.Function("H", [theta, gamma, sigma_w, sigma_s0], [H], ["theta", "gamma", "sigma_w", "sigma_s0"], ["H"])

# Plotting

## 1. Expected cumulative reward and the gradient 

In [ ]:
# Define range of theta values for plotting
theta_num = np.linspace(0.4, 0.9, 1000)

# Set fixed parameters for the environment
gamma_num = 0.9
sigma_w_num = 0.1 ** 0.5
sigma_s0_num = 0.1 ** 0.5

# Prepare rootfinder dictionary for finding optimal theta
rf_dict = {
    "g": grad_J,  # Gradient of J with respect to theta (exact)
    "x": theta,   # Variable to solve for (theta^\star)
    "p": cd.vertcat(*[gamma, sigma_w, sigma_s0]),  # Parameters
}

# Create rootfinder object using Newton's method
rf_theta_star = cd.rootfinder("grad_J", "newton", rf_dict)

# Find optimal theta (where grad_J = 0)
theta_star = rf_theta_star(x0 = cd.DM(0.75), p = cd.DM([gamma_num, sigma_w_num, sigma_s0_num]))["x"]

# Evaluate expected cumulative reward and its gradient for plotting
# Here we use the exact gradient and the Gauss-Newton matrix
J_num = J_func(theta_num, gamma_num, sigma_w_num, sigma_s0_num)
J_star = J_func(theta_star, gamma_num, sigma_w_num, sigma_s0_num)

# Evaluate the gradient of J via the deterministic policy gradient theorem
grad_J_num = grad_J_func(theta_num, gamma_num, sigma_w_num, sigma_s0_num)
grad_J_RL_num = grad_J_RL_func(theta_num, gamma_num, sigma_w_num, sigma_s0_num)
grad_J_RL_star = grad_J_RL_func(theta_star, gamma_num, sigma_w_num, sigma_s0_num)

# Plot results: J(theta) and gradients
nrows = 2
fig, ax = plt.subplots(nrows = nrows, figsize=(5, 4 * nrows), sharex=True, constrained_layout=True)

# Plot expected cumulative reward
ax[0].plot(theta_num, J_num, color='blue', label =r"$J(\theta)$")
ax[0].plot(theta_star, J_star, marker = "*", markersize = 10, linestyle = "None", color='red', label = r"$J(\theta^\star)$")
ax[0].set_ylabel(r"$J(\theta)$")
ax[0].set_xlim(min(theta_num), max(theta_num))

# Plot gradients
ax[1].plot(theta_num, grad_J_num, color='blue', label=r"$\nabla J(\theta)$")
ax[1].plot(theta_num, grad_J_RL_num, color='red', linestyle = "dashed", label=r"$\mathbb{E}_{s}[\nabla_{\theta}\pi_\theta(s) \nabla Q(s, a)\vert_{a=\pi_{\theta}(s)}]$")
ax[1].plot(theta_star, grad_J_RL_star, marker = "*", markersize = 10, color='red', linestyle = "None", label = r"$J(\theta^\star)$")

ax[1].set_ylabel(r"$\nabla J(\theta)$")
ax[1].legend()

# Add grid to all subplots
for axis in ax:
    axis.grid()

ax[-1].set_xlabel(r"$\theta$")

## 2. Hessians

In [ ]:
# Compute the exact Hessian, approximate Hessian, and Gauss-Newton matrix for a range of theta values
hess_J_num = hess_J_func(theta_num, gamma_num, sigma_w_num, sigma_s0_num) # This is the exact Hessian
hess_J_star = hess_J_func(theta_star, gamma_num, sigma_w_num, sigma_s0_num)

H_num = H_func(theta_num, gamma_num, sigma_w_num, sigma_s0_num) # This is the approximate Hessian (M1 + M2)
H_star = H_func(theta_star, gamma_num, sigma_w_num, sigma_s0_num)

Gauss_Newton_matrix_num = Gauss_Newton_matrix_func(theta_num, gamma_num, sigma_w_num, sigma_s0_num) # This is the Gauss-Newton matrix (M2)
Gauss_Newton_matrix_star = Gauss_Newton_matrix_func(theta_star, gamma_num, sigma_w_num, sigma_s0_num)

# Plot the Hessians and Gauss-Newton matrix
k = 1
fig, ax = plt.subplots(1, 1, figsize = (k * 5, k* 3.5), constrained_layout=True)

# Plot the exact Hessian, the approximate Hessian (M1 + M2), and the Gauss-Newton matrix (M2)
ax.plot(theta_num, hess_J_num, linestyle = "solid", label=r"Exact Hessian $\nabla_{\theta}^2 J(\theta)$", color="tab:red")
ax.plot(theta_num, H_num, linestyle = "dashed", label=r"Approx. Hessian $M_1(\theta) + M_2(\theta)$", color="tab:blue")
ax.plot(theta_num, Gauss_Newton_matrix_num, linestyle = "-.", label=r"Gauss-Newton $M_2(\theta)$ (Proposed)", color="tab:orange")

# Mark the value at the optimal parameter theta*
ax.plot(theta_star, hess_J_star, marker = "*", markersize = 15, linestyle = "None", color='red', label = r"At optimal parameter $\theta^\star$")

# Set axis labels and limits
ax.set_xlabel(r"$\theta$")
ax.set_ylabel(r"$\nabla_{\theta}^2 J(\theta)$")
ax.grid()
ax.set_xlim(min(theta_num), max(theta_num))
ax.set_ylim([-25, 3])

# Move the legend inside the plot to avoid it being cut off
ax.legend(loc="lower right", ncol=1, fontsize=11.5, frameon=True)

# Save the figure in high resolution
fig.savefig("Hessian_investigation.png", bbox_inches='tight', dpi=1200.0)


# Optimization with Newton, approx. Newton, Gauss-Newton, and Gradient ascent

## 1. Parameters

In [ ]:
# Initial value for theta
theta_init = 0.6

# Number of optimization steps
n_steps = 20

## 2. Newton's method with exact Hessian

In [ ]:
# --- Newton's method with exact Hessian ---
alpha_exact_hessian = 1.0
theta_init_full_hessian = theta_init
theta_full_hessian_list = [theta_init_full_hessian]

for idx in range(n_steps):
    # Compute gradient and Hessian of J at current theta
    grad_J_full_hessian = grad_J_func(theta_init_full_hessian, gamma_num, sigma_w_num, sigma_s0_num)
    hess_J_full_hessian = hess_J_func(theta_init_full_hessian, gamma_num, sigma_w_num, sigma_s0_num)

    # Newton update: theta <- theta - H^{-1} * grad
    theta_init_full_hessian -= alpha_exact_hessian * np.linalg.inv(hess_J_full_hessian) @ grad_J_full_hessian

    # Store updated theta
    theta_full_hessian_list.append(theta_init_full_hessian[0, 0])

# Stack results for plotting
theta_full_hessian = np.vstack(theta_full_hessian_list)

## 3. Newton's method with approximate Hessian ($M_1 + M_2$)

In [ ]:
# --- Newton's method with approximate Hessian (M1 + M2) ---
alpha_approx_hessian = 1.0
theta_init_approx_newton = theta_init
theta_approx_newton_list = [theta_init_approx_newton]

for idx in range(n_steps):
    # Compute gradient and approximate Hessian of J
    grad_J_approx_newton = grad_J_func(theta_init_approx_newton, gamma_num, sigma_w_num, sigma_s0_num)
    H_approx_newton = H_func(theta_init_approx_newton, gamma_num, sigma_w_num, sigma_s0_num)

    # Newton update with approximate Hessian
    theta_init_approx_newton -= alpha_approx_hessian * np.linalg.inv(H_approx_newton) @ grad_J_approx_newton

    # Store updated theta
    theta_approx_newton_list.append(theta_init_approx_newton[0, 0])

theta_approx_newton = np.vstack(theta_approx_newton_list)

## 4. Newton's method with Gauss-Newton approximation ($M_2$)

In [ ]:
# --- Gauss-Newton method (M2 only) ---
alpha_Gauss_Newton = 1.0
theta_init_Gauss_Newton = theta_init
theta_Gauss_Newton_list = [theta_init_Gauss_Newton]

for idx in range(n_steps):
    # Compute gradient and Gauss-Newton matrix
    grad_J_Gauss_Newton = grad_J_func(theta_init_Gauss_Newton, gamma_num, sigma_w_num, sigma_s0_num)
    Gauss_Newton_matrix = Gauss_Newton_matrix_func(theta_init_Gauss_Newton, gamma_num, sigma_w_num, sigma_s0_num)

    # Newton update with Gauss-Newton matrix
    theta_init_Gauss_Newton -= alpha_Gauss_Newton * np.linalg.inv(Gauss_Newton_matrix) @ grad_J_Gauss_Newton

    # Store updated theta
    theta_Gauss_Newton_list.append(theta_init_Gauss_Newton[0, 0])

theta_Gauss_Newton = np.vstack(theta_Gauss_Newton_list)

# 5. Gradient ascent

In [ ]:
# --- Gradient ascent ---
alpha_gradient_ascent = 0.1
theta_init_gradient_ascent = theta_init
theta_gradient_ascent_list = [theta_init_gradient_ascent]

for idx in range(n_steps):
    # Compute gradient of J
    grad_J_gradient_ascent = grad_J_func(theta_init_gradient_ascent, gamma_num, sigma_w_num, sigma_s0_num)

    # Gradient ascent update: theta <- theta + alpha * grad
    theta_init_gradient_ascent += alpha_gradient_ascent * grad_J_gradient_ascent

    # Store updated theta
    theta_gradient_ascent_list.append(theta_init_gradient_ascent[0, 0])

theta_gradient_ascent = np.vstack(theta_gradient_ascent_list)

# Plot the error dynamics

In [ ]:
import matplotlib.ticker as ticker

# Compute absolute errors between iterates and optimal theta for each method
full_hessian_error = np.abs(theta_full_hessian - theta_star)
approx_newton_error = np.abs(theta_approx_newton - theta_star)
Gauss_Newton_error = np.abs(theta_Gauss_Newton - theta_star)
gradient_ascent_error = np.abs(theta_gradient_ascent - theta_star)

# Plot convergence errors for all methods
fig, ax = plt.subplots(1, 1, figsize=(5, 3.5), constrained_layout=True)

# Plot error for Newton's method with exact Hessian, approximate Hessian, Gauss-Newton method, and gradient ascent 
ax.plot(np.arange(n_steps + 1), full_hessian_error, label=r"Exact Hessian $\nabla_{\theta}^2J(\theta)$", color="tab:red")
ax.plot(np.arange(n_steps + 1), approx_newton_error, label=r"Approx. Hessian $M_1(\theta) + M_2(\theta)$", color="tab:blue", linestyle="dashed")
ax.plot(np.arange(n_steps + 1), Gauss_Newton_error, label=r"Gauss-Newton $M_2(\theta)$ (Proposed)", color="tab:orange", linestyle ="-.")
ax.plot(np.arange(n_steps + 1), gradient_ascent_error, label="Gradient ascent", color="tab:green", linestyle="dotted")

# Set axis labels and formatting
ax.set_xlabel(r"Iteration $k$")
ax.set_ylabel(r"$| \theta_k - \theta^\star|$")
ax.set_yscale("log")  # Log scale for error
ax.set_ylim([1e-10, 1e0])
ax.set_xlim(0, n_steps)
ax.legend(loc="upper right", ncol=1, fontsize=11.5, frameon=True)
ax.grid()
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))  # Show integer ticks only

# Save the figure as a high-resolution PNG
fig.savefig("theta_convergence_error.png", bbox_inches='tight', dpi=1200.0)